## Uploading Raw Data to a SQL .db File 1

This is the code I used to create a .db file for some of our raw input data

Use this file as team reference for future SQL creations

For this particular file, I will upload raw unmodified data from FIRMS (satellite detections from NASA) and SCAN (soil moisture data from USDA)

**FIRMS** (Fire Information Resource Management System) exports of satellite fire dections can be optained in the following link:
https://firms.modaps.eosdis.nasa.gov/download/

**USDA** report exports of stations that track soil moisture, wind, humidity etc. can be found at the folowing link:
https://wcc.sc.egov.usda.gov/reportGenerator/
Note that the export comes with some headers at the top of the csv you have to manually remove first

**WFIGS** reports of actual wildfires and relevant data. This was pulled from the following link: https://data-nifc.opendata.arcgis.com/datasets/wfigs-wildland-fire-perimeters-full-history/explore?location=-0.000000%2C0.000000%2C0.00&showTable=true . You can find descriptions of these columns in the following link: https://www.arcgis.com/home/item.html?id=2191f997056547bd9dc530ab9866ab61 .

3 tables will be created for this .db file:
- **firms**: FIRMS MODIS data from Jan 1, 2020 to Dec 31, 2021
- **usda**: Selected USDA data from Jan 1, 2020 to April 6, 2022
- **wfigs**: Full history of WFIGS pulled on April 25, 2022 from link above

The tables will be created by merging the following files from my local computer for team to reference on S3 and DataGrip:
- **fire_archive_M-C61_258142.csv**: Part 1 of FIRMS Jan 1, 2020 to Dec 31, 2021
- **fire_nrt_M-C61_258142.csv**: Part 2 of FIRMS Jan 1, 2020 to Dec 31, 2021
- **SCANSelectedData2020.csv**: USDA of soil moisture and other relevant fields I exported for 2020
- **SCANSelectedData2021.csv**: USDA of soil moisture and other relevant fields I exported for 2021
- **SCANSelectedData2022_ToApril6.csv**: USDA of soil moisture and other relevant fields I exported for Jan 1, 2022 to April 6, 2022
- **WFIGS_-_Wildland_Fire_Perimeters_Full_History 4_25_22.csv**: The WFIGS data pulled on April 25, 2022

-Ben

In [1]:
#general imports
import pandas as pd
import sqlite3 
pd.set_option('display.max_columns', 200)

In [2]:
#read the MODIS csv's on my local PC
dfMODISa = pd.read_csv(r'C:\Users\anderb4\Documents\GeorgetownDSCert\WildfireCapstone\Data\Modis20to22\fire_archive_M-C61_258142.csv')
dfMODISb = pd.read_csv(r'C:\Users\anderb4\Documents\GeorgetownDSCert\WildfireCapstone\Data\Modis20to22\fire_nrt_M-C61_258142.csv')
dfMODIS = pd.concat([dfMODISa, dfMODISb]) #concatonate the data 
print(dfMODIS.shape)
dfMODIS.head()

(317580, 15)


,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,31.8349,-83.0561,306.7,1.4,1.2,2020-01-01,324,Terra,MODIS,69,6.03,277.6,18.9,N,0.0
1,36.3562,-76.8050,300.2,1.2,1.1,2020-01-01,325,Terra,MODIS,20,6.03,278.9,10.2,N,0.0
2,33.4178,-110.8616,321.5,1.0,1.0,2020-01-01,920,Aqua,MODIS,100,6.03,271.8,24.9,N,2.0
3,41.4810,-90.8294,310.0,1.0,1.0,2020-01-01,1704,Terra,MODIS,69,6.03,276.5,13.8,D,0.0
4,38.6973,-90.1281,311.7,1.1,1.0,2020-01-01,1705,Terra,MODIS,54,6.03,282.3,14.1,D,2.0


All reliable FIRMS from MODIS I have for the time being is now in the pandas df 'dfMODIS'

Now moving on to putting the SCAN data in a a pandas df:

In [3]:
#now read the USDA exports 
df_SCAN2020 = pd.read_csv(r'C:\Users\anderb4\Documents\GeorgetownDSCert\WildfireCapstone\GetSoilMoisture\SCANSelectedData2020.csv')
df_SCAN2021 = pd.read_csv(r'C:\Users\anderb4\Documents\GeorgetownDSCert\WildfireCapstone\GetSoilMoisture\SCANSelectedData2021.csv')
dfSCAN = pd.concat([df_SCAN2020, df_SCAN2021]) #concatonate the scan data
print(dfSCAN.shape)
dfSCAN.columns = dfSCAN.columns.str.replace(' ', '') #removes spaces in column names for better sql conversion
dfSCAN.head()

(133443, 37)


,Date,StationId,StateCode,NetworkCode,StationName,Elevation(ft),Latitude,Longitude,CountyName,HUC2(2-digit),HUC2Name,HUC4(4-digit),HUC4Name,HUC12(12-digit),HUC12Name,StartDate,EndDate,SnowDepth(in)StartofDayValues,PrecipitationAccumulation(in)StartofDayValues,PrecipitationIncrement(in),PrecipitationMonth-to-date(in)StartofDayValues,SoilMoisturePercent-2in(pct)StartofDayValues,SoilMoisturePercent-4in(pct)StartofDayValues,SoilMoisturePercent-6in(pct)StartofDayValues,SoilMoisturePercent-8in(pct)StartofDayValues,SoilMoisturePercent-10in(pct)StartofDayValues,DewPointTemperature(degF),RelativeHumidity(pct)StartofDayValues,RelativeHumidityAverage(pct),RelativeHumidityEnclosure(pct),WindMovementTotal(mile),WindSpeedAverage324in(mph),WindSpeedAverage128in(mph),WindSpeedAverage(mph),WindSpeedMaximum324in(mph),WindSpeedMaximum128in(mph),WindSpeedMaximum(mph)
0,1/1/2020,2057,AL,SCAN,AAMU-JTG,860,34.78333,-86.55,Madison,6.0,Tennessee Region,603.0,Middle Tennessee-Elk,6.030002e+10,Acuff Spring-Flint River,2/23/2002,1/1/2100,NaN,NaN,NaN,NaN,29.5,35.8,NaN,40.2,NaN,NaN,NaN,NaN,92.0,NaN,NaN,NaN,2.2,NaN,NaN,NaN
1,1/2/2020,2057,AL,SCAN,AAMU-JTG,860,34.78333,-86.55,Madison,6.0,Tennessee Region,603.0,Middle Tennessee-Elk,6.030002e+10,Acuff Spring-Flint River,2/23/2002,1/1/2100,NaN,NaN,NaN,NaN,28.9,35.4,NaN,40.2,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN
2,1/3/2020,2057,AL,SCAN,AAMU-JTG,860,34.78333,-86.55,Madison,6.0,Tennessee Region,603.0,Middle Tennessee-Elk,6.030002e+10,Acuff Spring-Flint River,2/23/2002,1/1/2100,NaN,NaN,NaN,NaN,37.8,40.1,NaN,41.4,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,2.4,NaN,NaN,NaN
3,1/4/2020,2057,AL,SCAN,AAMU-JTG,860,34.78333,-86.55,Madison,6.0,Tennessee Region,603.0,Middle Tennessee-Elk,6.030002e+10,Acuff Spring-Flint River,2/23/2002,1/1/2100,NaN,NaN,NaN,NaN,34.4,39.2,NaN,41.5,NaN,NaN,NaN,NaN,96.0,NaN,NaN,NaN,4.6,NaN,NaN,NaN
4,1/5/2020,2057,AL,SCAN,AAMU-JTG,860,34.78333,-86.55,Madison,6.0,Tennessee Region,603.0,Middle Tennessee-Elk,6.030002e+10,Acuff Spring-Flint River,2/23/2002,1/1/2100,NaN,NaN,NaN,NaN,31.5,36.7,NaN,40.5,NaN,NaN,NaN,NaN,91.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN


Now 2 the FIRMS and SCAN dataframes exist ('dfMODIS' and 'dfSCAN' respectively)

Now creating the dataframe for the WFIGS reports:

In [4]:
dfWFIGS = pd.read_csv(r'C:\Users\anderb4\Documents\GeorgetownDSCert\WildfireCapstone\Data\WFIGS_Pulled4_25_22\WFIGS_-_Wildland_Fire_Perimeters_Full_History 4_25_22.csv')
print(dfWFIGS.shape)
dfWFIGS.head()

(10049, 108)


C:\Users\anderb4\AppData\Local\Temp\ipykernel_6236\481921373.py:1: DtypeWarning: Columns (40,69,72) have mixed types. Specify dtype option on import or set low_memory=False.
  dfWFIGS = pd.read_csv(r'C:\Users\anderb4\Documents\GeorgetownDSCert\WildfireCapstone\Data\WFIGS_Pulled4_25_22\WFIGS_-_Wildland_Fire_Perimeters_Full_History 4_25_22.csv')


,OBJECTID,poly_IncidentName,poly_FeatureCategory,poly_MapMethod,poly_GISAcres,poly_CreateDate,poly_DateCurrent,poly_PolygonDateTime,poly_Acres_AutoCalc,poly_GlobalID,poly_Source,irwin_ABCDMisc,irwin_ADSPermissionState,irwin_CalculatedAcres,irwin_ContainmentDateTime,irwin_ControlDateTime,irwin_DailyAcres,irwin_DiscoveryAcres,irwin_DispatchCenterID,irwin_EstimatedCostToDate,irwin_FFReportApprovedByTitle,irwin_FFReportApprovedByUnit,irwin_FFReportApprovedDate,irwin_FireBehaviorGeneral,irwin_FireBehaviorGeneral1,irwin_FireBehaviorGeneral2,irwin_FireBehaviorGeneral3,irwin_FireCause,irwin_FireCauseGeneral,irwin_FireCauseSpecific,irwin_FireCode,irwin_FireDepartmentID,irwin_FireDiscoveryDateTime,irwin_FireMgmtComplexity,irwin_FireOutDateTime,irwin_FSConfinePercent,irwin_FSFullSuppPercent,irwin_FSMonitorPercent,irwin_FSPointZonePercent,irwin_FSJobCode,irwin_FSOverrideCode,irwin_GACC,irwin_ICS209ReportDateTime,irwin_ICS209RForTimePeriodFrom,irwin_ICS209RForTimePeriodTo,irwin_ICS209ReportStatus,irwin_IncidentManagementOrg,irwin_IncidentName,irwin_IncidentShortDescription,irwin_IncidentTypeCategory,irwin_IncidentTypeKind,irwin_InitialLatitude,irwin_InitialLongitude,irwin_InitialResponseAcres,irwin_InitialResponseDateTime,irwin_IrwinID,irwin_IsFireCauseInvestigated,irwin_IsFireCodeRequested,irwin_IsFSAssisted,irwin_IsMultiJurisdictional,irwin_IsReimbursable,irwin_IsTrespass,irwin_IsUnifiedCommand,irwin_LocalIncidentIdentifier,irwin_PercentContained,irwin_PercentPerToBeContained,irwin_POOCity,irwin_POOCounty,irwin_POODispatchCenterID,irwin_POOFips,irwin_POOJurisdictionalAgency,irwin_POOJurisdictionalUnit,irwin_POOJurisdUnitParentUnit,irwin_POOLandownerCategory,irwin_POOLandownerKind,irwin_POOLegalDescPrinMeridian,irwin_POOLegalDescQtr,irwin_POOLegalDescQtrQtr,irwin_POOLegalDescRange,irwin_POOLegalDescSection,irwin_POOLegalDescTownship,irwin_POOPredServiceAreaID,irwin_POOProtectingAgency,irwin_POOProtectingUnit,irwin_POOState,irwin_PredominantFuelGroup,irwin_PredominantFuelModel,irwin_PrimaryFuelModel,irwin_SecondaryFuelModel,irwin_TotalIncidentPersonnel,irwin_UniqueFireIdentifier,irwin_WFDSSDecisionStatus,irwin_CreatedBySystem,irwin_ModifiedBySystem,irwin_IsDispatchComplete,irwin_OrganizationalAssessment,irwin_StratDecisionPublishDate,irwin_GlobalID,irwin_Source,irwin_ArchivedOn,irwin_ModifiedOnDateTime_dt,irwin_CreatedOnDateTime_dt,GlobalID,irwin_IsCpxChild,irwin_CpxName,irwin_CpxID,SHAPE_Length,SHAPE_Area
0,127,Coleman,Wildfire Daily Fire Perimeter,Mixed Methods,562.913504,2021/05/04 22:50:38+00,2021/05/04 22:50:38+00,NaN,562.911767,{951569B8-EA6B-443B-8BF7-022AE1DFC937},2020 NIFS,NaN,DEFAULT,574.424,2020/11/15 23:00:00+00,2020/12/31 22:15:00+00,574.0,1.0,CALPCC,3925000.0,NaN,NaN,NaN,Minimal,Smoldering,Creeping,Isolated Torching,Unknown,NaN,NaN,NPD9,NaN,2020/10/18 19:24:00+00,Type 4 Incident,2021/03/10 16:15:00+00,2.0,98.0,0.0,0.0,P5,0507,OSCC,2020/11/20 00:00:00+00,2020/11/19 14:00:00+00,2020/11/20 00:00:00+00,F,Type 4 IC,COLEMAN,Coleman Reservoir Area,WF,FI,36.07114,-121.4505,3.0,NaN,{DB0B9472-74B9-4D89-8E78-A792C26FE4F2},NaN,0,0.0,0.0,0.0,1.0,0.0,003161,100.0,100.0,King City,Monterey,CALPCC,06053,FS,CALPF,NaN,USFS,Federal,Mount Diablo,NaN,NaN,06E,17.0,21S,SC07,FS,CALPF,US-CA,NaN,NaN,Brush (2 feet),Tall Grass (2.5 feet),1.0,2020-CALPF-003161,Approved Decision,wildcad,wildcad,0,Type 3 Incident,2020/10/20 14:14:11+00,NaN,IRWIN,NaN,2021/03/26 19:01:00+00,2020/10/18 20:02:59+00,{1E48F205-2208-4DBD-8A48-B739FDEACF9C},NaN,NaN,NaN,0.120036,-2.280138e-04
1,128,Patrick,Wildfire Daily Fire Perimeter,Mixed Methods,0.151680,2021/05/04 22:51:03+00,2021/05/04 22:51:03+00,NaN,0.151680,{08D5DE2B-51A0-420A-812F-35902565E93A},2020 NIFS,NaN,CERTIFIED,NaN,2020/05/01 20:39:00+00,2020/05/01 20:39:00+00,0.1,0.1,NVSFC,NaN,NaN,NaN,2020/06/19 20:31:59+00,NaN,NaN,NaN,NaN,Unknown,Cause and Origin Not Identified,NaN,M26Z,NaN,2020/05/01 18:56:59+00,NaN,NaN,NaN,NaN,NaN,NaN,PN,1502,GBCC,NaN,NaN,NaN,NaN,NaN,Patrick,NaN,WF,FI,39.55669,-119.5585,NaN,NaN,{8C96BC5C-225D-49

In [5]:
#the warning above looks weird, checking the names of those columns:
mixedcols = []
mixedcols.append(dfWFIGS.columns[40])
mixedcols.append(dfWFIGS.columns[69])
mixedcols.append(dfWFIGS.columns[72])
print(mixedcols)

['irwin_FSOverrideCode', 'irwin_POOFips', 'irwin_POOJurisdUnitParentUnit']


In [6]:
#convert those columns to strings for consistency to address the warning 
for columnname in mixedcols:
    dfWFIGS[[columnname]] = dfWFIGS[[columnname]].astype(str) 

We now have all 3 dataframes we eventually want to convert to a table. 

Now create the .db file and connect to it:

In [7]:
# create .db file and start cursor 
dbpath = 'modis_and_scan_raw1.db' # names the db
conn = sqlite3.connect(dbpath) # creates the db
cursor = conn.cursor() # cursor for modifications and viewing 

Now create a table for FIRMS:

In [8]:
sql = """CREATE TABLE IF NOT EXISTS firms(
            latitude REAL,
            longitude REAL,
            scan REAL,
            track REAL,
            acq_date REAL,
            acq_time INTEGER,
            satellite TEXT,
            instrument TEXT,
            confidence INTEGER,
            version REAL,
            bright_t31 REAL,
            frp REAL,
            daynight TEXT,
            type REAL
        );"""

cursor.execute(sql)

In [9]:
for row in dfMODIS.itertuples():
    sql = """INSERT INTO firms (latitude, longitude, scan, track, acq_date, acq_time, satellite, instrument, confidence, version, bright_t31, frp, daynight, type)
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)
            """
    cursor.execute(sql,(row.latitude, row.longitude, row.scan, row.track, row.acq_date, row.acq_time, row.satellite, row.instrument, row.confidence, row.version, row.bright_t31, row.frp, row.daynight, row.type))

conn.commit()

In [10]:
#verify that the database is updated for firms 
sql = """SELECT * FROM firms
        LIMIT 3"""
cursor.execute(sql)
cursor.fetchall()

[(31.8349,
  -83.0561,
  1.4,
  1.2,
  '2020-01-01',
  324,
  'Terra',
  'MODIS',
  69,
  6.03,
  277.6,
  18.9,
  'N',
  0.0),
 (36.3562,
  -76.805,
  1.2,
  1.1,
  '2020-01-01',
  325,
  'Terra',
  'MODIS',
  20,
  6.03,
  278.9,
  10.2,
  'N',
  0.0),
 (33.4178,
  -110.8616,
  1.0,
  1.0,
  '2020-01-01',
  920,
  'Aqua',
  'MODIS',
  100,
  6.03,
  271.8,
  24.9,
  'N',
  2.0)]

It appears like it is working. I am now going to try and use the pandas to_sql functions now for WFIGS:

In [11]:
#converts df to a table in the .db object
dfWFIGS.to_sql(name='wfigs', con = conn)

10049

In [12]:
#verify that the wfigs table was created 
sql = """SELECT * FROM wfigs
        LIMIT 1"""
cursor.execute(sql)
cursor.fetchall()
 

[(0,
  127,
  'Coleman',
  'Wildfire Daily Fire Perimeter',
  'Mixed Methods',
  562.913504426158,
  '2021/05/04 22:50:38+00',
  '2021/05/04 22:50:38+00',
  None,
  562.911767294885,
  '{951569B8-EA6B-443B-8BF7-022AE1DFC937}',
  '2020 NIFS',
  None,
  'DEFAULT',
  574.424,
  '2020/11/15 23:00:00+00',
  '2020/12/31 22:15:00+00',
  574.0,
  1.0,
  'CALPCC',
  3925000.0,
  None,
  None,
  None,
  'Minimal',
  'Smoldering',
  'Creeping',
  'Isolated Torching',
  'Unknown',
  None,
  None,
  'NPD9',
  None,
  '2020/10/18 19:24:00+00',
  'Type 4 Incident',
  '2021/03/10 16:15:00+00',
  2.0,
  98.0,
  0.0,
  0.0,
  'P5',
  '0507',
  'OSCC',
  '2020/11/20 00:00:00+00',
  '2020/11/19 14:00:00+00',
  '2020/11/20 00:00:00+00',
  'F',
  'Type 4 IC',
  'COLEMAN',
  'Coleman Reservoir Area',
  'WF',
  'FI',
  36.07114,
  -121.4505,
  3.0,
  None,
  '{DB0B9472-74B9-4D89-8E78-A792C26FE4F2}',
  None,
  0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  '003161',
  100.0,
  100.0,
  'King City',
  'Monterey',
  'C

In [13]:
#seems like it worked, going to double check the column names got transfered over 
sql = """SELECT poly_IncidentName FROM wfigs
        LIMIT 10"""
cursor.execute(sql)
cursor.fetchall()

[('Coleman',),
 ('Patrick ',),
 ('Low Top',),
 ('Sloan 2',),
 ('Bloody nose',),
 ('Beach',),
 ('Lava Rock',),
 ('KUTCHER RIDGE',),
 ('MCCLUNG',),
 ('Sawdust',)]

Seems like it worked for WFIGS. 
Now doing the same thing for USDA dataframe and converting that to the db object:

In [14]:
dfSCAN.to_sql(name='usda', con = conn)

133443

In [15]:
#verify that it went into the .db file accuratley 
sql = """SELECT * FROM usda
        LIMIT 3"""
cursor.execute(sql)
cursor.fetchall()

[(0,
  '1/1/2020',
  2057,
  'AL',
  'SCAN',
  'AAMU-JTG',
  860,
  34.78333,
  -86.55,
  'Madison',
  6.0,
  'Tennessee Region',
  603.0,
  'Middle Tennessee-Elk',
  60300020403.0,
  'Acuff Spring-Flint River',
  '2/23/2002',
  '1/1/2100',
  None,
  None,
  None,
  None,
  29.5,
  35.8,
  None,
  40.2,
  None,
  None,
  None,
  None,
  92.0,
  None,
  None,
  None,
  2.2,
  None,
  None,
  None),
 (1,
  '1/2/2020',
  2057,
  'AL',
  'SCAN',
  'AAMU-JTG',
  860,
  34.78333,
  -86.55,
  'Madison',
  6.0,
  'Tennessee Region',
  603.0,
  'Middle Tennessee-Elk',
  60300020403.0,
  'Acuff Spring-Flint River',
  '2/23/2002',
  '1/1/2100',
  None,
  None,
  None,
  None,
  28.9,
  35.4,
  None,
  40.2,
  None,
  None,
  None,
  None,
  100.0,
  None,
  None,
  None,
  3.0,
  None,
  None,
  None),
 (2,
  '1/3/2020',
  2057,
  'AL',
  'SCAN',
  'AAMU-JTG',
  860,
  34.78333,
  -86.55,
  'Madison',
  6.0,
  'Tennessee Region',
  603.0,
  'Middle Tennessee-Elk',
  60300020403.0,
  'Acuff Spring

In [19]:
#seems like it is working, going to double check the column names are there as well
sql = """SELECT StationName FROM usda
        LIMIT 10 OFFSET 1000"""
cursor.execute(sql)
cursor.fetchall()

[('Adams Ranch #1',),
 ('Adams Ranch #1',),
 ('Adams Ranch #1',),
 ('Adams Ranch #1',),
 ('Adams Ranch #1',),
 ('Adams Ranch #1',),
 ('Adams Ranch #1',),
 ('Adams Ranch #1',),
 ('Adams Ranch #1',),
 ('Adams Ranch #1',)]

In [20]:
conn.close()

### The SQL .db is now created with all the data intended to put on it

This includes the firms, wfigsd, and usda tables. The sql .db file will be available on the team's S3. 

-Ben